# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [1]:
import pandas as pd

# Load data
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

# Filter data sesuai instruksi
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]

# Tampilkan hasil
filtered_df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [3]:
# Filter hanya pelanggan dengan response "Yes"
df_yes = df[df['Response'] == 'Yes']

# Group by policy type dan gender, lalu hitung rata-rata total claim
average_claim = df_yes.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean().round(2)

# Tampilkan hasil
average_claim

Policy Type     Gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: Total Claim Amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [5]:
# Hitung total customer per state
customers_per_state = df['State'].value_counts()

# Filter state yang jumlahnya lebih dari 500
states_above_500 = customers_per_state[customers_per_state > 500]

# Tampilkan hasil
states_above_500

State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
# Gunakan groupby untuk education dan gender
clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median'])

# Tampilkan hasil
clv_stats

max          min       median
Education            Gender                                       
Bachelor             F       73225.95652  1904.000852  5640.505303
                     M       67907.27050  1898.007675  5548.031892
College              F       61850.18803  1898.683686  5623.611187
                     M       61134.68307  1918.119700  6005.847375
Doctor               F       44856.11397  2395.570000  5332.462694
                     M       32677.34284  2267.604038  5577.669457
High School or Below F       55277.44589  2144.921535  6039.553187
                     M       83325.38119  1940.981221  6286.731006
Master               F       51016.06704  2417.777032  5729.855012
                     M       50568.25912  2272.307310  5579.099207

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [9]:
# Step 1: Pastikan kolom tanggal dalam format datetime
df['effective_to_date'] = pd.to_datetime(df['Effective To Date'])

# Step 2: Ambil nama bulan
df['month'] = df['effective_to_date'].dt.month_name()

# Step 3: Buat pivot table
policies_by_state_month = pd.pivot_table(
    df,
    values='Number of Policies',
    index='State',
    columns='month',
    aggfunc='sum',
    fill_value=0
)

# Tampilkan hasil
policies_by_state_month.head()

C:\Users\Heri Soesanto\AppData\Local\Temp\ipykernel_9824\1410687860.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effective_to_date'] = pd.to_datetime(df['Effective To Date'])


month,February,January
State,,
Arizona,2864,3052
California,4929,5673
Nevada,1278,1493
Oregon,3969,4697
Washington,1225,1358


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [11]:
# Step 1: Pastikan kolom tanggal dalam format datetime
df['effective_to_date'] = pd.to_datetime(df['Effective To Date'])

# Step 2: Ambil nama bulan
df['month'] = df['effective_to_date'].dt.month_name()

# Step 3: Buat pivot table
policies_by_state_month = pd.pivot_table(
    df,
    values='Number of Policies',
    index='State',
    columns='month',
    aggfunc='sum',
    fill_value=0
)

# Tampilkan hasil
policies_by_state_month.head()

C:\Users\Heri Soesanto\AppData\Local\Temp\ipykernel_9824\1410687860.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effective_to_date'] = pd.to_datetime(df['Effective To Date'])


month,February,January
State,,
Arizona,2864,3052
California,4929,5673
Nevada,1278,1493
Oregon,3969,4697
Washington,1225,1358


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [15]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type', 'effective_to_date', 'month'],
      dtype='object')

In [17]:
# Step 1: Hitung total pelanggan per Sales Channel
total_by_channel = df.groupby('Sales Channel')['Response'].value_counts().unstack()

# Step 2: Hitung persentase "Yes" dari total
total_by_channel['Response Rate (% Yes)'] = (
    total_by_channel['Yes'] / 
    (total_by_channel['Yes'] + total_by_channel['No'])
) * 100

# Step 3: Bulatkan
total_by_channel = total_by_channel.round(2)

# Tampilkan hasilnya
total_by_channel

Response,No,Yes,Response Rate (% Yes)
Sales Channel,,,
Agent,3148,742,19.07
Branch,2539,326,11.38
Call Center,1792,221,10.98
Web,1334,177,11.71
